In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [2]:
import bokeh.layouts as bkl
import bokeh.plotting as bk
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

# Load result files

In [3]:
import turbo_broccoli as tb
import regex as re
from pathlib import Path

SWEEP_DIRS = [
    Path("out") / "sweep",
    Path("out") / "baselines",
]
SWEEP_RESULTS = []

for sweep_dir in SWEEP_DIRS:
    for p in sweep_dir.glob("**/results.*.json"):
        if not re.match(r"results\.\w+\.json", p.name):
            continue
        try:
            print("Loading", p)
            data = tb.load(p)
            SWEEP_RESULTS.append(data)
        except Exception as e:
            print("ERROR: Couldn't load", p, ":", type(e), str(e))

print()
print("Loaded", len(SWEEP_RESULTS), "results")

Loading out/sweep/cifar100/microsoft-resnet-18/results.0.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.1.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.2.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.3.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.4.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.5.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.6.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.7.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.8.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.9.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.10.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.11.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.12.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.13.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.14.json
Loading out/sweep/cifar100/microsoft-resnet-18/results.15.json
Lo

# Gather baselines

`baselines` is a dict

    dataset -> model_name -> ["acc1" or "acc5"] -> a float

In [4]:
from collections import defaultdict
import numpy as np

BASELINE_OVERRIDES = {}

baselines = defaultdict(dict)
for r in SWEEP_RESULTS:
    if r["model"]["hparams"]["lcc_kwargs"] is not None:
        continue
    dataset, model = r["dataset"]["name"], r["model"]["name"]
    if model not in baselines[dataset]:
        baselines[dataset][model] = {"acc1": [], "acc5": []}
    baselines[dataset][model]["acc1"].append(r["training"]["test"][0]["test/acc"])
    baselines[dataset][model]["acc5"].append(
        r["training"]["test"][0].get("test/acc5", 1)
    )

for bls in baselines.values():
    for model, d in bls.items():
        bls[model]["acc1"] = np.max(bls[model]["acc1"])
        bls[model]["acc5"] = np.max(bls[model]["acc5"])

baselines

defaultdict(dict,
            {'cifar100': {'microsoft/resnet-18': {'acc1': 0.785099983215332,
               'acc5': 0.9606000185012817},
              'alexnet': {'acc1': 0.714900016784668,
               'acc5': 0.920799970626831},
              'timm/tinynet_e.in1k': {'acc1': 0.743399977684021, 'acc5': 1}}})

# Gather all results (incl. baselines) in a dataframe

In [5]:
import pandas as pd
import numpy as np

from sweep import LCC_INTERVALS, LCC_WARMUPS, LCC_WEIGHTS, LCC_KS
from sweep import MODELS as _MODELS
from sweep import DATASETS as _DATASETS

MODELS = set(d["name"] for d in _MODELS)
DATASETS = set(d["name"] for d in _DATASETS)

GAIN_THRESHOLD = -10  # in percent

data = []

for ds, bsls in baselines.items():
    if ds not in DATASETS:
        continue
    for model, d in bsls.items():
        if model not in MODELS:
            continue
        data.append(
            {
                "model": model,
                "dataset": ds,
                "test_acc1": d["acc1"],
                "test_acc5": d["acc5"],
                "gain1": 0,
                "gain5": 0,
            }
        )

for r in SWEEP_RESULTS:
    if r["model"]["name"] not in MODELS or r["dataset"]["name"] not in DATASETS:
        # print(
        #     "Run not in model list or dataset list:",
        #     r["model"]["name"],
        #     r["dataset"]["name"],
        # )
        continue
    lcc_kwargs = r["model"]["hparams"]["lcc_kwargs"]
    lcc_submodules = r["model"]["hparams"].get("lcc_submodules")
    if lcc_submodules is None:  # Baseline run
        continue
    if not (
        lcc_kwargs is None
        or (
            lcc_kwargs["weight"] in LCC_WEIGHTS
            and lcc_kwargs["interval"] in LCC_INTERVALS
            and lcc_kwargs["warmup"] in LCC_WARMUPS
            and lcc_kwargs["k"] in LCC_KS
        )
    ):
        # print("Run LCC parameters now in sweep")
        continue
    if hasattr(lcc_submodules, "__len__") and len(lcc_submodules) > 1:
        # print("Run has more than one LCC submodule")
        continue

    row = {
        "model": r["model"]["name"],
        "dataset": r["dataset"]["name"],
        "test_acc1": r["training"]["test"][0]["test/acc"],
    }
    if (a5 := r["training"]["test"][0].get("test/acc5")) is not None:
        row["test_acc5"] = a5
    if lcc_kwargs is None:
        row["gain1"], row["gain5"] = 0, 0
    else:
        row["submodule"] = ",".join(lcc_submodules)
        row.update(lcc_kwargs)
        try:
            bacc1 = baselines[row["dataset"]][row["model"]]["acc1"]
            gain = row["test_acc1"] - bacc1
            row["gain1"] = gain
        except KeyError:
            row["gain1"] = 0
        try:
            bacc5 = baselines[row["dataset"]][row["model"]]["acc5"]
            gain = row["test_acc5"] - bacc5
            row["gain5"] = gain
        except KeyError:
            row["gain5"] = 0

    data.append(row)

df = pd.DataFrame(data)
df = df.drop(
    ["test_acc5", "gain5", "ccspc", "clustering_method", "loss", "interval"], axis=1
)
# df = df.sort_values(columns, na_position="first")
# df

In [6]:
columns = ["model", "dataset", "weight", "k", "interval", "warmup", "submodule"]
columns = [s for s in columns if s in df.columns]
df = df.groupby(columns, dropna=False).mean()
df = df.reset_index()
df = df.sort_values(columns, na_position="first")

interesting_columns = [
    c
    for c in df.columns
    if (df[c].isna().any() and len(df[c].unique()) > 2)
    or (not df[c].isna().any() and len(df[c].unique()) > 1)
    or c.startswith("test_acc")
    or c.startswith("gain")
]
if nic := [c for c in df.columns if c not in interesting_columns]:
    print("Dropping", nic)
    df = df.drop(nic, axis=1)

df

Dropping ['dataset', 'warmup']


,model,weight,k,submodule,test_acc1,gain1
12,alexnet,NaN,NaN,NaN,0.714900,0.000000
0,alexnet,0.0001,5.0,classifier,0.713133,-0.001767
1,alexnet,0.0001,5.0,classifier.4,0.713900,-0.001000
2,alexnet,0.0001,50.0,classifier,0.712867,-0.002033
3,alexnet,0.0001,50.0,classifier.4,0.714133,-0.000767
4,alexnet,0.0001,500.0,classifier,0.711900,-0.003000
5,alexnet,0.0001,500.0,classifier.4,0.709100,-0.005800
6,alexnet,0.0100,5.0,classifier,0.716950,0.002050
7,alexnet,0.0100,5.0,classifier.4,0.714750,-0.000150
8,alexnet,0.0100,50.0,classifier,0.717183,0.002283


# Tex export

In [7]:
N_DECIMALS = 2


def float_to_latex(x: float | str) -> str:
    if isinstance(x, str):
        return x
    if np.isnan(x):
        return ""
    float_str = f"{x:.2E}"
    if "E" in float_str:
        base, exponent = float_str.split("E")
        base, exponent = int(float(base)), int(exponent)
        if exponent < 0:
            if base == 1:
                return f"$10^{{{exponent}}}$"
            return f"${base} \\times 10^{{{exponent}}}$"
        return "$" + str(int(base * 10**exponent)) + "$"
    return "$" + str(round(x, N_DECIMALS)) + "$"


def int_to_latex(x: float | str) -> str:
    if isinstance(x, str):
        return x
    if np.isnan(x):
        return ""
    return "$" + str(int(x)) + "$"


def percent_to_latex(x: float | str, sign: bool = False) -> str:
    if isinstance(x, str):
        return x
    if np.isnan(x):
        return ""
    return (
        "$" + ("+" if sign and x > 0 else "") + str(round(x * 100, N_DECIMALS)) + "\\%$"
    )


data = []
for _, row in df.iterrows():

    row_fmt = {}

    if dataset := row.get("dataset"):
        if dataset == "cifar100":
            dataset = "CIFAR100"
        elif dataset == "timm/imagenet-1k-wds":
            dataset = "ImageNet"
        elif dataset == "timm/eurosat-rgb":
            dataset = "EuroSat"
        elif dataset == "timm/oxford-iiit-pet":
            dataset = "Oxford-IIIT Pet"
        elif dataset == "timm/resisc45":
            dataset = "RESISC45"
        else:
            print("Unknown dataset:", dataset, "Skipping.")
            continue
        row_fmt["dataset"] = dataset

    if model := row.get("model"):
        model = row["model"]
        if model.startswith("google/mobilenet") or model.startswith("timm/mobilenet"):
            model = "MobileNet"
        elif model.startswith("microsoft/resnet-18") or model.startswith(
            "timm/resnet18"
        ):
            model = "ResNet18"
        elif model.startswith("google/vit-base-patch16-224"):
            model = "ViT"
        elif model.startswith("timm/tinynet"):
            model = "TinyNet"
        elif model.startswith("timm/vgg11"):
            model = "VGG11"
        elif model.startswith("timm/tf_efficientnet_l2"):
            model = "EfficientNet"
        elif model.startswith("timm/convnext_small"):
            model = "ConvNeXt"
        elif model.startswith("timm/inception_v3"):
            model = "InceptionV3"
        elif model == "alexnet":
            model = "AlexNet"
        else:
            print("Unknown model:", model, "Skipping.")
            continue
        row_fmt["Model"] = model

    layer = row.get("submodule")
    if layer is None or not isinstance(layer, str):
        layer = ""
    elif layer in ["classifier", "head", "fc"]:
        layer = "Head"
    else:
        layer = "$2^{\\text{nd}}$ to last"
        # layer = "$\\texttt{" + str(layer).replace("_", "\_") + "}$"
    row_fmt["Layer"] = layer

    if weight := row.get("weight"):
        row_fmt["$w$"] = float_to_latex(weight)

    if k := row.get("k"):
        row_fmt["$k$"] = int_to_latex(k)

    if (wmp := row.get("warmup")) is not None:
        row_fmt["Wmp."] = int_to_latex(row["warmup"])

    if interval := row.get("interval"):
        row_fmt["Int."] = int_to_latex(row["interval"])

    acc_1 = percent_to_latex(row["test_acc1"], sign=False)
    gain_1 = percent_to_latex(row["gain1"], sign=True) if row["gain1"] else ""
    if "test_acc5" in df.columns:
        acc_5 = percent_to_latex(row["test_acc5"], sign=False)
        gain_5 = percent_to_latex(row["gain5"], sign=True) if row["gain5"] else ""
        row_fmt.update(
            {
                "Acc. (top 1)": acc_1,
                "Gain (top 1)": gain_1,
                "Acc. (top 5)": acc_5,
                "Gain (top 5)": gain_5,
            }
        )
    else:
        row_fmt.update({"Acc.": acc_1, "Gain": gain_1})
    data.append(row_fmt)

df_tex = pd.DataFrame(data)

In [8]:
from IPython.display import display

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(df_tex)

,Model,Layer,$w$,$k$,Acc.,Gain
0,AlexNet,,,,$71.49\%$,
1,AlexNet,Head,$10^{-4}$,$5$,$71.31\%$,$-0.18\%$
2,AlexNet,$2^{\text{nd}}$ to last,$10^{-4}$,$5$,$71.39\%$,$-0.1\%$
3,AlexNet,Head,$10^{-4}$,$50$,$71.29\%$,$-0.2\%$
4,AlexNet,$2^{\text{nd}}$ to last,$10^{-4}$,$50$,$71.41\%$,$-0.08\%$
5,AlexNet,Head,$10^{-4}$,$500$,$71.19\%$,$-0.3\%$
6,AlexNet,$2^{\text{nd}}$ to last,$10^{-4}$,$500$,$70.91\%$,$-0.58\%$
7,AlexNet,Head,$10^{-2}$,$5$,$71.69\%$,$+0.2\%$
8,AlexNet,$2^{\text{nd}}$ to last,$10^{-2}$,$5$,$71.48\%$,$-0.02\%$
9,AlexNet,Head,$10^{-2}$,$50$,$71.72\%$,$+0.23\%$


In [9]:
from pathlib import Path

OUTPUT_PATH = Path("out/papers/")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

In [10]:
with (OUTPUT_PATH / "results-all.tex").open("w", encoding="utf-8") as fp:
    df_tex.to_latex(fp, index=False)

for model in df_tex["Model"].unique():
    with (OUTPUT_PATH / f"results-{model.lower()}.tex").open(
        "w", encoding="utf-8"
    ) as fp:
        a = df_tex[df_tex["Model"] == model]
        a.drop("Model", axis=1).to_latex(fp, index=False)

# Typst export

Reference: https://typst.app/docs/reference/model/table

In [11]:
import re

header = [f"[*{c}*]" for c in data[0]]
n_columns = len(header)

typst = f"""
#figure(table(
  columns: ({', '.join(['auto'] * n_columns)}),
  table.hline(),
  table.header(
    {', '.join(header)}
  ),
  table.hline(stroke: .5pt),"""

for d in data:
    typst += "\n// ------------------------------"
    for k, v in d.items():
        if m := re.match(r"\$10\^{(.*)}\$", v):
            v = f"$10^({m.group(1)})$"
        elif m := re.match(r"\$(.*)\^{\\text{(.*)}}\$(.*)", v):
            a, b, c = m.groups()
            v = f'${a}^("{b}")${c}'
        v = v.replace("\\%", "%")
        typst += f"\n  [{v}], // {k}"

typst += "\n))"
print(typst)


#figure(table(
  columns: (auto, auto, auto, auto, auto, auto),
  table.hline(),
  table.header(
    [*Model*], [*Layer*], [*$w$*], [*$k$*], [*Acc.*], [*Gain*]
  ),
  table.hline(stroke: .5pt),
// ------------------------------
  [AlexNet], // Model
  [], // Layer
  [], // $w$
  [], // $k$
  [$71.49%$], // Acc.
  [], // Gain
// ------------------------------
  [AlexNet], // Model
  [Head], // Layer
  [$10^(-4)$], // $w$
  [$5$], // $k$
  [$71.31%$], // Acc.
  [$-0.18%$], // Gain
// ------------------------------
  [AlexNet], // Model
  [$2^("nd")$ to last], // Layer
  [$10^(-4)$], // $w$
  [$5$], // $k$
  [$71.39%$], // Acc.
  [$-0.1%$], // Gain
// ------------------------------
  [AlexNet], // Model
  [Head], // Layer
  [$10^(-4)$], // $w$
  [$50$], // $k$
  [$71.29%$], // Acc.
  [$-0.2%$], // Gain
// ------------------------------
  [AlexNet], // Model
  [$2^("nd")$ to last], // Layer
  [$10^(-4)$], // $w$
  [$50$], // $k$
  [$71.41%$], // Acc.
  [$-0.08%$], // Gain
// -------------

In [12]:
with (OUTPUT_PATH / "results-all.typ").open("w", encoding="utf-8") as fp:
    fp.write(typst)